In [1]:
import cartopy.io.shapereader as shpreader
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np
import pandas as pd
import os
import sys
from osgeo import gdal,ogr, osr
import xarray
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

Topographic data: SRTM Digital Elevation Data Version 4
* https://developers.google.com/earth-engine/datasets/catalog/CGIAR_SRTM90_V4

In [8]:
## Mosaic data
data_path = "/Volumes/ChenLab/Fujiang/0_PhD_dissertation_data/8_topographic_data/1_original_data/"
out_path = "/Volumes/ChenLab/Fujiang/0_PhD_dissertation_data/8_topographic_data/2_mosaic_data/"
file_lists = os.listdir(data_path)

variables = ["Elevation", "Slope", "Aspect"]

for var in variables:
    data_list = [f"{data_path}{x}" for x in file_lists if var in x and ".tif" in x]
    out_tif = f"{out_path}{var}.tif"
    
    options = gdal.WarpOptions(format='GTiff')
    mosaic = gdal.Warp(out_tif, data_list, options=options)
    mosaic = None 
    print("Mosaic created successfully at:", out_tif)

Mosaic created successfully at: /Volumes/ChenLab/Fujiang/0_PhD_dissertation_data/8_topographic_data/1_original_data/Elevation.tif
Mosaic created successfully at: /Volumes/ChenLab/Fujiang/0_PhD_dissertation_data/8_topographic_data/1_original_data/Slope.tif
Mosaic created successfully at: /Volumes/ChenLab/Fujiang/0_PhD_dissertation_data/8_topographic_data/1_original_data/Aspect.tif


In [ ]:
## Mosaic data (Alaska)
data_path = "/Volumes/ChenLab/Fujiang/0_PhD_dissertation_data/8_topographic_data/1_original_data/Alaska/"
out_path = "/Volumes/ChenLab/Fujiang/0_PhD_dissertation_data/8_topographic_data/2_mosaic_data/"
file_lists = os.listdir(data_path)

variables = ["Elevation_Alaska"]

for var in variables:
    data_list = [f"{data_path}{x}" for x in file_lists if var in x and ".tif" in x]
    out_tif = f"{out_path}{var}.tif"
    
    options = gdal.WarpOptions(format='GTiff')
    mosaic = gdal.Warp(out_tif, data_list, options=options)
    mosaic = None 
    print("Mosaic created successfully at:", out_tif)

In [13]:
## clip

def read_tif(tif_file):
    dataset = gdal.Open(tif_file)
    cols = dataset.RasterXSize
    rows = dataset.RasterYSize
    im_proj = (dataset.GetProjection())
    im_Geotrans = (dataset.GetGeoTransform())
    im_data = np.moveaxis(dataset.ReadAsArray(0, 0, cols, rows), 0, -1)
    return im_data, im_Geotrans, im_proj,rows, cols

prisma_path = "/Volumes/ChenLab/Fujiang/0_PhD_dissertation_data/2_PRISMA_L2D/2_PRISMA_L2D_tif_2020_2023/"
data_path = "/Volumes/ChenLab/Fujiang/0_PhD_dissertation_data/8_topographic_data/2_mosaic_data/"
out_path = "/Volumes/ChenLab/Fujiang/0_PhD_dissertation_data/8_topographic_data/3_clipped_NEON_extent/"

variables = ["Elevation", "Slope", "Aspect"]
site_list = ['D01_BART','D01_HARV','D02_SCBI','D03_OSBS','D07_MLBS','D07_ORNL','D08_TALL',
             'D10_CPER','D13_MOAB','D14_JORN','D14_SRER','D16_WREF','D19_BONA','D19_HEAL']

for site in site_list:
    path = f"{prisma_path}{site}"
    file_list = os.listdir(path)
    file_list = [x for x in file_list if "LATLON.tif" in x]

    ul_x_all,lr_y_all, lr_x_all, ul_y_all = [],[],[],[]
    for file in file_list:
        im_data, im_Geotrans, im_proj,rows, cols = read_tif(f"{path}/{file}")
        ul_x, lr_y, lr_x, ul_y = im_data[0,0,1],im_data[-1,-1,0],im_data[-1,-1,1],im_data[0,0,0]
        ul_x_all.append(ul_x)
        lr_y_all.append(lr_y)
        lr_x_all.append(lr_x)
        ul_y_all.append(ul_y)
    ul_x, lr_y, lr_x, ul_y = min(ul_x_all),min(lr_y_all), max(lr_x_all), max(ul_y_all)
    ul_x, lr_y, lr_x, ul_y = ul_x-0.2, lr_y-0.2, lr_x+0.2, ul_y+0.2
    print(site, ul_x, lr_y, lr_x, ul_y)
    
    os.makedirs(f"{out_path}{site}", exist_ok=True)
    for var in variables:
        file = f"{var}.tif"
        input_tif = f"{data_path}{file}"
        output_tif =f"{out_path}{site}/{file}"
        gdal.Warp(output_tif, input_tif, format = 'GTiff', outputBounds=(ul_x, lr_y, lr_x, ul_y))
        input_tif = None
        output_tif = None
        print(site,var,datetime.now().strftime("%Y-%m-%d %H:%M:%S"),"finished")
        
        ### transfer the projection of clipped data
        in_tif = f"{out_path}{site}/{file}"

        out_tif = f"{out_path}{site}/{file}"
        input_ds = gdal.Open(in_tif)
        output_ds = gdal.Warp(out_tif, input_ds, dstSRS=im_proj)
        input_ds = None
        output_ds = None

D01_BART -71.75350494384766 43.562035369873044 -70.66015319824218 44.545073699951175
D01_BART Elevation 2024-07-12 14:07:31 finished
D01_BART Slope 2024-07-12 14:07:39 finished
D01_BART Aspect 2024-07-12 14:07:46 finished
D01_HARV -72.74523468017578 42.03336029052734 -71.72100524902343 43.04200286865235
D01_HARV Elevation 2024-07-12 14:07:59 finished
D01_HARV Slope 2024-07-12 14:08:06 finished
D01_HARV Aspect 2024-07-12 14:08:14 finished
D02_SCBI -78.57472534179688 38.52046661376953 -77.71292572021484 39.27803726196289
D02_SCBI Elevation 2024-07-12 14:08:22 finished
D02_SCBI Slope 2024-07-12 14:08:28 finished
D02_SCBI Aspect 2024-07-12 14:08:34 finished
D03_OSBS -82.44445343017578 29.2985408782959 -81.57926635742187 30.057671737670898
D03_OSBS Elevation 2024-07-12 14:08:53 finished
D03_OSBS Slope 2024-07-12 14:08:59 finished
D03_OSBS Aspect 2024-07-12 14:09:04 finished
D07_MLBS -81.1591064453125 36.89331893920898 -80.1043212890625 37.90365142822266
D07_MLBS Elevation 2024-07-12 14:09:1